In [174]:
import pandas as pd
pd.set_option('display.precision',15)
import re
import os
cwdir = os.getcwd()
cwdir = cwdir+'/'

In [175]:
df_npa = pd.read_csv(cwdir+'fordb.csv', index_col=0)
df_master = pd.read_csv(cwdir+'Master\Master.csv')

In [176]:
df_master['Phone'] = df_master['Phone'].astype(str)          
df_master['Phone'] = df_master['Phone'].str.replace('.0', ' ')
df_master['Practice Phone'] = df_master['Practice Phone'].astype(str)          
df_master['Practice Phone'] = df_master['Practice Phone'].str.replace('.0', ' ')
df_npa['Provider Business Mailing Address Telephone Number'] = df_npa['Provider Business Mailing Address Telephone Number'].astype(str)          
df_npa['Provider Business Mailing Address Telephone Number'] = df_npa['Provider Business Mailing Address Telephone Number'].str.replace('.0', ' ')
df_npa['Provider Business Practice Location Address Telephone Number'] =df_npa['Provider Business Practice Location Address Telephone Number'].astype(str)          
df_npa['Provider Business Practice Location Address Telephone Number'] =df_npa['Provider Business Practice Location Address Telephone Number'].str.replace('.0', ' ')

df_npa = df_npa.dropna(axis=1, how='all')

In [178]:
df_npa=df_npa.loc[df_npa['Provider Business Practice Location Address City Name'] == 'CORAL SPRINGS']
#df_npa['Provider Business Practice Location Address City Name']
df_master=df_master.loc[df_master['Practice City']== 'CORAL SPRINGS']
df_master['Practice City']

df_master.to_csv(cwdir+'Demo/CoralSpringsData_master.csv')
df_npa.to_csv(cwdir+'Demo/CoralSpringsData_NPI.csv')

In [179]:
df_npa = df_npa.sort_values("NPI")
df_master['flag'] = None

In [180]:
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import re
import jellyfish
pd.set_option('display.precision',15)
from pyjarowinkler import distance

cols_practice_npa_dup= ['Provider First Line Business Practice Location Address', 'Provider Second Line Business Practice Location Address', 'Provider Business Practice Location Address City Name']
cols_practice_npa= ['Provider First Line Business Practice Location Address', 'Provider Second Line Business Practice Location Address']
cols_practice_master = ['Practice Street Address 1', 'Practice Street Address 2']
df_master['check1'] = df_master[cols_practice_master].apply(lambda x: ','.join(x.dropna()), axis=1)
df_npa['check1'] = df_npa[cols_practice_npa].apply(lambda x: ','.join(x.dropna()), axis=1)
cols_practice_npa = ['Provider First Line Business Mailing Address','Provider Second Line Business Mailing Address']
cols_practice_master = ['Shipping Address Line 1', 'Shipping Address Line 2', 'Shipping Address Line 3']
df_master['check2'] = df_master[cols_practice_master].apply(lambda x: ','.join(x.dropna()), axis=1)
df_npa['check2'] = df_npa[cols_practice_npa].apply(lambda x: ','.join(x.dropna()), axis=1)
df_npa['dupcheck1'] = df_npa[cols_practice_npa_dup].apply(lambda x: ','.join(x.dropna()), axis=1)
df_npa = df_npa.drop_duplicates(subset =['dupcheck1'], 
                     keep = 'first') 

df_npa = df_npa.drop(['dupcheck1'],axis=1)


In [181]:
index_jf=[]
for index,row in df_npa.iterrows():
    f1 = re.sub('[^a-zA-Z0-9]', '', row['Provider Organization Name (Legal Business Name)'])
    for index1,row1 in df_master.iterrows():
        f2 = re.sub('[^a-zA-Z0-9]', '', row1['Account Name'])
             #check3 = re.sub('[^a-zA-Z0-9]', '', row['fullname'])
            #check4 = re.sub('[^a-zA-Z0-9]', '', row1['fullname'])
        if jellyfish.jaro_winkler(f1,f2)>.90:
            print(f1,f2)
            index_jf.append(index)
            if jellyfish.jaro_winkler(row['check1'],row1['check1'])>.90:
                print(row['check1'],row1['check1'])
                df_master.at[index1,'flag']='No Action'
                
            

EYECONTACTWESTINC EYECONTACTWEST
10657 WILES RD 10657 WILES RD
ADVANTAGEVISIONCARE ADVANTAGEVISIONCARELL
11312 WILES RD 11312 WILES RD
CORALSPRINGSVISIONCAREPLLC CORALSPRINGSVISIONSOURCE
9773 W SAMPLE RD 9773 W SAMPLE RD


In [182]:
df_npa=df_npa.drop(index_jf)

In [183]:
index_uf=[]
for index,row in df_npa.iterrows():
    f1 = re.sub('[^a-zA-Z0-9]', '', row['check1'])
    for index1,row1 in df_master.iterrows():
        f2 = re.sub('[^a-zA-Z0-9]', '', row1['check1'])
             #check3 = re.sub('[^a-zA-Z0-9]', '', row['fullname'])
            #check4 = re.sub('[^a-zA-Z0-9]', '', row1['fullname'])
        if jellyfish.jaro_winkler(f1,f2)>.90 and row['Provider Business Practice Location Address Telephone Number']==row1['Practice Phone']:
            index_uf.append(index)   
            print(row['Provider Organization Name (Legal Business Name)'],row1['Account Name'])
            print(row['Provider Business Practice Location Address Telephone Number'],row1['Practice Phone'])
            df_master.at[index1,'Account Name']=row['Provider Organization Name (Legal Business Name)']
            df_master.at[index1,'flag']='Account Name Updated'
        
df_master['flag']         

MARK E GOLDBERG OD PA DR MARK GOLDBERG
95475295  95475295 
AMANDIP SINGH SAPPAL 2, O.D., P.A. SHERIDAN EYE CARE
95475537  95475537 
AMANDIP SINGH SAPPAL 2, O.D., P.A. SHERIDAN EYE CARE
95475537  95475537 
CHERYL A. LENNARD, PA CITY EYECARE
9547521553 9547521553


4675                     None
5991     Account Name Updated
8589                     None
10973                    None
16080                    None
19814                    None
20795                    None
21237                    None
22511                    None
22556    Account Name Updated
26114                    None
27686                    None
28935                    None
31403                    None
32929                    None
38003                    None
41041                    None
41298    Account Name Updated
41583    Account Name Updated
41957                    None
44582                    None
45391                    None
46588                    None
Name: flag, dtype: object

In [184]:
df_npa=df_npa.drop(index_uf)

In [185]:
index_vf=[]
for index,row in df_npa.iterrows():
    f1 = re.sub('[^a-zA-Z0-9]', '', row['check1'])
    for index1,row1 in df_master.iterrows():
        f2 = re.sub('[^a-zA-Z0-9]', '', row1['check1'])
             #check3 = re.sub('[^a-zA-Z0-9]', '', row['fullname'])
            #check4 = re.sub('[^a-zA-Z0-9]', '', row1['fullname'])
        if jellyfish.jaro_winkler(f1,f2)>.90:
            index_vf.append(index)   
            df_master.at[index1,'Account Name']=row['Provider Organization Name (Legal Business Name)']
            df_master.at[index1,'Practice Phone'] = row['Provider Business Practice Location Address Telephone Number']
            df_master.at[index1,'flag']='Phone and AccountName Updated'

df_master['flag']


4675     Phone and AccountName Updated
5991              Account Name Updated
8589     Phone and AccountName Updated
10973                             None
16080                             None
19814                             None
20795    Phone and AccountName Updated
21237                             None
22511                             None
22556             Account Name Updated
26114                             None
27686    Phone and AccountName Updated
28935    Phone and AccountName Updated
31403                             None
32929                             None
38003    Phone and AccountName Updated
41041                             None
41298             Account Name Updated
41583             Account Name Updated
41957                             None
44582                             None
45391                             None
46588    Phone and AccountName Updated
Name: flag, dtype: object

In [186]:
df_npa=df_npa.drop(index_vf)
df_npa

,Entity Type Code,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Other Organization Name,Provider Other Organization Name Type Code,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code,...,Healthcare Provider Taxonomy Group_9,Healthcare Provider Taxonomy Group_10,Healthcare Provider Taxonomy Group_11,Healthcare Provider Taxonomy Group_12,Healthcare Provider Taxonomy Group_13,Healthcare Provider Taxonomy Group_14,Healthcare Provider Taxonomy Group_15,Certification Date,check1,check2
NPI,,,,,,,,,,,,,,,,,,,,,
1023444536,2.0,<UNAVAIL>,"DANIEL, MAX & MARCANDREA",EYELAB,3.0,3801 S CONGRESS AVE,NaN,PALM SPRINGS,FL,334614140.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,741 N UNIVERSITY DR,3801 S CONGRESS AVE
1134420029,2.0,<UNAVAIL>,"ICUC OPTOMETRIST, PC",NaN,NaN,3860 VICTORY BLVD,NaN,STATEN ISLAND,NY,103146720.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,432 NW 105TH DR,3860 VICTORY BLVD
1265631022,2.0,<UNAVAIL>,CARLA PARIS TEICH O D P L,NaN,NaN,5794 NW 48TH DR,NaN,CORAL SPRINGS,FL,330674015.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5794 NW 48TH DR,5794 NW 48TH DR
1437492527,2.0,<UNAVAIL>,"RICHARD A. BERMAN, O.D., P.A.",NaN,NaN,4635 N UNIVERSITY DR,NaN,CORAL SPRINGS,FL,330674602.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4635 N UNIVERSITY DR,4635 N UNIVERSITY DR
1649576059,2.0,<UNAVAIL>,JOSEALIX DAVERMAN OD PA,NaN,NaN,6841 MIRAMAR PKWY,NaN,MIRAMAR,FL,330236023.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5920 W SAMPLE RD # 105-7,6841 MIRAMAR PKWY


In [188]:
for index,row in df_npa.iterrows():
    f1 = re.sub('[^a-zA-Z0-9]', '', row['Provider Organization Name (Legal Business Name)'])
    p1=re.sub('[^a-zA-Z0-9]', '', row['Provider Business Practice Location Address Telephone Number'])
    for index1,row1 in df_master.iterrows():
        f2 = re.sub('[^a-zA-Z0-9]', '', row1['Account Name'])
        p2 = re.sub('[^a-zA-Z0-9]', '', row1['Practice Phone'])
        
        if jellyfish.jaro_winkler(f1,f2)>.80 and jellyfish.jaro_winkler(p1,p2)>.90:
            print(p1,p2)
df_master.to_csv(cwdir+'Demo/updatedMasterData.csv')  
df_npa.to_csv(cwdir+'Demo/npiinsertdata.csv')
         